This code book calls the OpenAI API to classify moral sentiments in posts from the Moral Foundations Reddit Corpus using ChatGPT (Fewshot)!

## Load Packages

In [1]:
import openai
import os
import pandas as pd
import numpy as np

import string
import re
remove = string.punctuation
remove = remove.replace("-", "").replace(",", "") # don't remove hyphens
pattern = r"[{}]".format(remove) # create the pattern

import pickle
import time
import logging
from retry import retry
logging.basicConfig()

# Calculate the delay based on your rate limit
rate_limit_per_minute = 3500.0
delay_60 = 60.0 / 60
delay_full = 60.0 / rate_limit_per_minute

## General Parameters

In [2]:
data = "mfrc"
mode = "full"
folder = "../data/preprocessed/"
path = folder + data + "_sample_" + mode + ".csv"
path_train = folder + data + "_train_" + mode + ".csv"

## Load Data

### Functions

In [3]:
def generate_prompts(definitions, examples, test_texts):
    prompts = {}
        
    for foundation, definition in definitions.items():
        full_instruction = "Determine the presence of \"{}\" in the following text. The text contains \"{}\" {}. Respond with \"yes\" if the sentiment is expressed in the text and \"no\" if it is not. Respond only with a single word and do not elaborate. Here is the text: ".format(foundation, foundation, definition)
        short_instruction = "Determine the presence of the moral sentiment of '{}' in the following text: ".format(foundation)
        
        example_texts = examples[foundation]
        messages = [{"role": "system", "content": SYSTEM_INSTR}]
        
        # For the first example, we use the full instruction
        first_example_text = example_texts[0]
        user_message = {"role": "user", "content": full_instruction + first_example_text}
        assistant_message = {"role": "assistant", "content": "yes"}
        messages.extend([user_message, assistant_message])
        
        # For subsequent examples, we use the shortened instruction
        for text in example_texts[1:]:
            user_message = {"role": "user", "content": short_instruction + text}
            assistant_message = {"role": "assistant", "content": "yes"}
            messages.extend([user_message, assistant_message])

        # For the test texts, we also use the shortened instruction
        for test_text in test_texts:
            user_test_message = {"role": "user", "content": short_instruction + test_text}
            full_prompt = {"messages": messages + [user_test_message]}
            prompts.setdefault(foundation, []).append(full_prompt)

    return prompts

# System role instructions
SYSTEM_INSTR = "Detect the presence of a moral sentiment in a text based on its provided definition."

FOUNDATIONS_DEFINITIONS = {
    "care": "if it is about avoiding emotional and physical damage to another individual",
    "equality": "if it is about equal treatment and equal outcome for individuals.",
    "proportionality": "if it is about individuals getting rewarded in proportion to their merit or contribution",
    "loyalty": "if it is about cooperating with ingroups and competing with outgroups",
    "authority": "if it is about deference toward legitimate authorities and the defense of traditions, all of which are seen as providing stability and fending off chaos",
    "purity": "if it is about avoiding bodily and spiritual contamination and degradation",
    "thin morality": "if it has a moral sentiment but cannot be categorized as a specific moral value"
}


### Generate Example Prompts for Fewshot Learning

In [4]:
# load training data
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path)
test_texts = df_test.text.tolist()

# create multiple df for each foundation
df_dict = {} # store training data for each prediction class here
for pred_class in df_train.columns[1:]: # separate data for each output
    df_dict[pred_class] = df_train[["text", pred_class]]
pred_classes = list(df_train.columns[1:])

In [7]:
# sample and manually validate 2 good examples
# other examples: care | equality| proportionality | loyalty | authority | purity | thin morality | non-moral |
# 3, 12, | 2, 18 | 6, 13, 14, 15 | 1, 3, 8 | 4, 7 | 6, 7, 9 | 0, 1, 6 | 10, 11, 17|
# find examples in df_df_dict
idx_dict = {"care": [25, 31], "equality": [10, 16], "proportionality": [12,  17], "loyalty": [0, 2], 
                 "authority": [1, 3], "purity": [0, 3], "thin morality": [2, 4], "non-moral": [1, 16]}
ex_dict = {key: np.array(df_train[(df_train[key] == 1) & (df_train.sum(1) == 1)].text.iloc[idx]) for key, idx in idx_dict.items()}

In [8]:
### Generate Prompts
prompts = generate_prompts(FOUNDATIONS_DEFINITIONS, ex_dict, test_texts)

In [9]:
# check prompts
prompts["care"][0]["messages"]

[{'role': 'system',
  'content': 'Detect the presence of a moral sentiment in a text based on its provided definition.'},
 {'role': 'user',
  'content': 'Determine the presence of "care" in the following text. The text contains "care" if it is about avoiding emotional and physical damage to another individual. Respond with "yes" if the sentiment is expressed in the text and "no" if it is not. Respond only with a single word and do not elaborate. Here is the text: &gt;she basically abused\n\nIt’s more like she literally abused him by depriving him of what constitutes a *limb*.'},
 {'role': 'assistant', 'content': 'yes'},
 {'role': 'user',
  'content': "Determine the presence of the moral sentiment of 'care' in the following text: 100% NTA. It absolutely is your place to get involved if you suspect animal abuse. If there is an explanation this will be uncovered and no real harm done except the owner may be a bit peeved. Alternatively, if they shouldn't be owning a dog, you'll be putting 

## API Call Functions

In [35]:
# chatGPT parameters
openai.api_key = os.getenv("OPENAI_API_KEY") # add your api key to the environment
model_engine = "gpt-3.5-turbo-0301"

@retry(delay=5)
def delayed_completion(delay_in_seconds: float = 1, **kwargs):
    """Delay a completion by a specified amount of time."""

    # Sleep for the delay
    time.sleep(delay_in_seconds)

    # Call the Completion API and return the result
    return openai.ChatCompletion.create(**kwargs)

def clean_response(x):
    return 1 if "yes" in x else 0

## Test Call

In [ ]:
# test all foundations with the first example
for foundation, test_calls in prompts.items():
    test_call = test_calls[0]["messages"]
    print(foundation)
    print(test_call)
    APIresponse = delayed_completion(
        delay_in_seconds=delay_full,
        model=model_engine,
        messages=test_call,
        temperature=0
        )
    response = APIresponse.choices[0].message["content"]
    print(response) #works

In [ ]:
# test a single example
test_call = prompts["purity"][61]["messages"]
print(test_call)

In [ ]:
print(test_call)
APIresponse = delayed_completion(
    delay_in_seconds=delay_full,
    model=model_engine,
    messages=test_call,
    temperature=0
    )
response = APIresponse.choices[0].message["content"]
print(response) #works

## Run Calls

In [11]:
new_dic = {}
new_dic["text"] = test_texts

for foundation, prompt_list in prompts.items():
    responses = []
    print(foundation)
    for i, prompt in enumerate(prompt_list):
        APIresponse = delayed_completion(
            delay_in_seconds=delay_full,
            model=model_engine,
            messages=prompt["messages"],
            temperature=0,
            )
        response = APIresponse.choices[0].message["content"]
        responses.append(response)
        if not i % int(0.1 * len(prompt_list)):
            print(str(int(i/len(prompt_list)*100)) + "\%")
    new_dic[foundation] = responses
    df_temp = pd.DataFrame(new_dic)
    df_temp.to_csv("../results/predictions/gpt_fewshot_" + data + "_labels_" + foundation + "_" + mode + ".csv", index=False)

# clean responses and save in final dataset
df_responses = pd.DataFrame(new_dic)
cols_to_clean = df_responses.columns[df_responses.columns != 'text']
df_responses[cols_to_clean] = df_responses[cols_to_clean].applymap(clean_response)
df_responses["non-moral"] = (df_responses.drop(columns=['text']).sum(axis=1) == 0).astype(int) # non-moral if no moral sentiment found
df_responses.to_csv("../results/predictions/gpt_fewshot_" + data + "_labels_" + mode + ".csv", index=False)

0\%
9\%
19\%
29\%
39\%
49\%
59\%
69\%
79\%
89\%
99\%
0\%
9\%
19\%
29\%
39\%
49\%
59\%
69\%
79\%
89\%


99\%
0\%
9\%
19\%
29\%
39\%
49\%
59\%
69\%
79\%
89\%
99\%
0\%
9\%
19\%
29\%
39\%
49\%
59\%
69\%
79\%
89\%
99\%
0\%
9\%
19\%
29\%
39\%
49\%
59\%
69\%
79\%
89\%
99\%
0\%
9\%
19\%
29\%
39\%
49\%
59\%
69\%
79\%
89\%
99\%
0\%
9\%
19\%
29\%
39\%
49\%
59\%
69\%
79\%
89\%
99\%
